# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [7]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [8]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_tsplib = pd.read_csv("experimentos/tsplib.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- DP: Programación dinámica.

In [22]:
def correr_experimento(metodo, archivo_instancia):
    print(metodo, archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", metodo, archivo_instancia], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [10]:
experimentos = [];

## Experimento 1
Correr Heurisitica Golosa sobre todas las instancias de tsplib.

In [11]:
for index, row in df_tsplib.iterrows():
    experimentos.append([row["instancia"], row["ruta_instancia"], "H-CG"]);

## Experimento 2
Correr BT para mejor y peor caso.

In [42]:
for n in range(0, df_mejor_caso_bt.shape[0]):
    fila_n = df_mejor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

for n in range(0, df_peor_caso_bt.shape[0]):
    fila_n = df_peor_caso_bt.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);

## Experimento 3
Correr BT, BT-F, BT-O para densidad alta y baja.

In [43]:
for n in range(0, df_densidad_alta.shape[0]):
    fila_n = df_densidad_alta.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

for n in range(0, 30):
    fila_n = df_densidad_baja.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-F", fila_n["archivo"]]);
    experimentos.append([fila_n["dataset"], n, fila_n["W"], "BT-O", fila_n["archivo"]]);

# Experimento 4
Correr DP para el dataset _dinamica_.

In [44]:
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

## Experimento 5
Correr DP para el dataset densidad-alta y densidad-baja.

In [45]:
for i in range(0, df_densidad_alta.shape[0]):
    fila = df_densidad_alta.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

for i in range(0, df_densidad_baja.shape[0]):
    fila = df_densidad_baja.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["W"], "DP", fila["archivo"]]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [23]:
columnas = ["dataset", "n", "W", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[1], experimento[2]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 1/110'

../../instancias/a280.txt H-CG


ValueError: could not convert string to float: 'Algoritmo no encontrado: ../../instancias/a280.txt\nLos algoritmos existentes son: \n\t- BL: Algoritmo de busqueda local\n\t- BL-Tabu: Algoritmo de busqueda local con Tabu Search\n\t- H-AGM: Heuristica basada en AGM\n\t- H-CG: Heuristica constructiva golosa\n'